# STEP 1: SETUP THE OPENAI API KEY AND BASE

In [1]:
# All Import Statements used in this step
import os
import openai

In [2]:
os.environ["OPENAI_API_KEY"] = # Enter your API Key
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [3]:
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OpenAI API Key not provided.")

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

In [5]:
MODEL = "gpt-3.5-turbo"

# STEP 2: GENERATE REAL ESTATE CSV USING LLM

In [7]:
# All Import statements used in this step
import openai

In [31]:
CSV_GENERATOR_SYSTEM_PROMPT = """
You are a CSV Data Generator.

RULES YOU MUST ADHERE TO WHEN CREATING CSV DATA:
1. Use a semi-colon (;) as the Delimiter, NOT a comma.
2. Do not number the rows.
3. No symbols like $, sqft, etc., are allowed within the CSV data.
4. All kinds of descriptions should be detailed and well-organized.

YOUR OUTPUT SHOULD ONLY CONSIST OF THE RAW CSV TEXT. DO NOT INCLUDE MARKDOWN BACKTICKS (```).
"""

In [32]:
QUERY_PROMPT = """
Generate at least 10 fictional Real Estates listings in CSV format.

The columns should be the following:
1. **neighborhood:** The neighborhood in which the property is .
2. **price:** The price of the property in USD.
3. **bedrooms:** The number of bedrooms present in the property. Should be a whole number between 1 to 5.
4. **bathrooms:** The number of bathrooms present in the property. Should be a whole number between 1 to 5.
5. **house_size:** The size of the property in sqft.
6. **description:** Detailed description of the property.
7. **neighborhood_description:** Detailed description of the neighborhood. 
"""

In [33]:
llm_response = openai.ChatCompletion.create(
    model=MODEL,
    temperature=0.7,
    messages=[
        {
            "role": "system",
            "content": CSV_GENERATOR_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": QUERY_PROMPT
        }
    ]
)

In [34]:
print(llm_response.choices[0].message.content)

neighborhood;price;bedrooms;bathrooms;house_size;description;neighborhood_description
Green Valley;350000;3;2;2000;Beautiful single-family home in the peaceful Green Valley neighborhood. This property features a spacious living room, modern kitchen, and a well-maintained backyard. Perfect for a small family looking for a cozy place to call home.;Green Valley is known for its lush greenery, friendly community, and top-rated schools. Residents enjoy easy access to parks, shopping centers, and dining options.
Downtown Heights;500000;2;2;1500;Luxurious condo located in the heart of Downtown Heights. This upscale property offers stunning city views, high-end finishes, and amenities such as a gym and rooftop terrace. Ideal for urban professionals seeking a stylish living space.;Downtown Heights is a vibrant neighborhood with an eclectic mix of restaurants, art galleries, and entertainment venues. It provides convenient access to public transportation and is perfect for those who enjoy city l

In [35]:
csv_data = llm_response.choices[0].message.content.replace("```csv", "").replace("```", "").strip()

In [36]:
with open("Real_Estates.csv", "w", encoding="utf-8") as file:
    file.write(csv_data)

# STEP 3: READ THE DATA FROM THE CSV

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(
    "Real_Estates.csv", 
    sep=";",
    engine="python"
)

In [8]:
df.head()

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Green Valley,350000,3,2,2000,Beautiful single-family home in the peaceful G...,"Green Valley is known for its lush greenery, f..."
1,Downtown Heights,500000,2,2,1500,Luxurious condo located in the heart of Downto...,Downtown Heights is a vibrant neighborhood wit...
2,Lakeview Estates,600000,4,3,2800,Spacious family home overlooking the serene La...,Lakeview Estates offers residents a peaceful r...
3,Sunset Hills,450000,3,2,1900,Charming ranch-style house nestled in the pict...,Sunset Hills is a sought-after neighborhood kn...
4,Riverfront Terrace,700000,5,4,3500,Elegant waterfront property located in the exc...,Riverfront Terrace is a prestigious neighborho...


# STEP 3: POPULATE VECTOR DATABASE

In [9]:
# All Import Statements used in this step
import shutil
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
CHROMA_PATH = "content/chroma-1"

In [11]:
embedding = OpenAIEmbeddings()

In [12]:
loader = DataFrameLoader(df, page_content_column="description")

In [13]:
documents = loader.load()
print(f"Successfully created {len(documents)} documents from CSV.")

Successfully created 10 documents from CSV.


In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True
)

In [15]:
chunks = text_splitter.split_documents(documents)
print(f"Successfully created {len(chunks)} chunks from {len(documents)} documents.")

Successfully created 10 chunks from 10 documents.


In [16]:
if chunks:
    document = chunks[5]
    print(document.page_content)

Modern hillside home offering panoramic views in the tranquil Hillside Retreat neighborhood. This contemporary property features an open floor plan, high ceilings, and a wrap-around deck for outdoor entertaining. Ideal for nature lovers seeking a peaceful escape from the city.


In [17]:
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

In [18]:
db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=CHROMA_PATH

)

In [19]:
db.persist()
print(f"Successfully populated vector database at {CHROMA_PATH}")

Successfully populated vector database at content/chroma-1


# STEP 4: SEMANTIC SEARCH USING RAG

In [20]:
# All Import statements in this step
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [21]:
llm = ChatOpenAI(
    model=MODEL,
    temperature=0
)

In [26]:
QA_TEMPLATE = """
You are LLMRealtor, a polite and friendly real estate agent.
Use the following context to recommend the best property to the buyer.

Context: {context}

Buyer Preference: {question}

Answer:
"""

In [27]:
QA_CHAIN_PROMPT = PromptTemplate.from_template(QA_TEMPLATE)

In [29]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [30]:
# Example User query
query = "A comfortable three-bedroom house with a spacious kitchen."

In [31]:
response = qa_chain.run(query)

In [32]:
print(response)

Based on your preferences for a comfortable three-bedroom house with a spacious kitchen, I would highly recommend the beautiful single-family home in the peaceful Green Valley neighborhood. This property offers a spacious living room, modern kitchen, and a well-maintained backyard, perfect for a small family looking for a cozy place to call home. It ticks all the boxes for comfort and functionality while also being located in a serene and family-friendly neighborhood. I believe this property would be a perfect fit for you and your family. Let me know if you would like to schedule a viewing!


# STEP 5: PERSONALIZED RECOMMENDATION USING LLM AND MEMORY CHAIN

In [33]:
# All Import Statements used in this step
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

In [34]:
llm = ChatOpenAI(model_name=MODEL, temperature=0.7)

In [35]:
memory = ConversationSummaryMemory(llm=llm)

In [36]:
realtor_template = """
You are a friendly AI Realtor.
Your goal is to gather information to build a buyer's profile.
Ask questions one by one about budget, location, bedrooms, and lifestyle preferences.
Do not recommend houses yet. Keep your responses short and conversational.

---

Current Conversation Summary:
{history}

User: {input}
Realtor:
"""

In [37]:
realtor_prompt = PromptTemplate(
    template=realtor_template,
    input_variables=["history", "input"]
)
print(realtor_prompt)

input_variables=['history', 'input'] template="\nYou are a friendly AI Realtor.\nYour goal is to gather information to build a buyer's profile.\nAsk questions one by one about budget, location, bedrooms, and lifestyle preferences.\nDo not recommend houses yet. Keep your responses short and conversational.\n\n---\n\nCurrent Conversation Summary:\n{history}\n\nUser: {input}\nRealtor:\n"


In [38]:
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=realtor_prompt,
    verbose=False
)

print("AI Realtor Initialized. Ready to Chat!")

AI Realtor Initialized. Ready to Chat!


In [39]:
print("--- STARTING INTERVIEW. (Type 'done' to finish) ---")

while True:
    user_input = input("You: ")

    if user_input.lower() in ["done", "exit", "quit", "finish"]:
        print("\n--- INTERVIEW FINISHED ---")
        break

    ai_response = conversation.predict(input=user_input)
    print(f"AI Realtor: {ai_response}")

--- STARTING INTERVIEW. (Type 'done' to finish) ---


You:  Hello. I am looking for a house. Can you help me?


AI Realtor: Of course! I'd be happy to help. Let's start by discussing your budget. What price range are you looking at for your new home?


You:  Something cheap. Under 800,000.


AI Realtor: Great! Keeping it affordable is important. Next, let's talk about the location you prefer. Do you have a specific area in mind where you'd like to live?


...

AI Realtor: No problem! How many bedrooms are you looking for in your new home?


You:  I am planning to have kids. So, at least 3.


AI Realtor: That's great to hear! Having at least 3 bedrooms for your growing family sounds like a good plan. Next, let's talk about your lifestyle preferences. Do you have any specific requirements or features you'd like in your new home?


You:  done



--- INTERVIEW FINISHED ---


In [40]:
summary_data = memory.load_memory_variables({})

In [42]:
user_profile_summary = summary_data["history"]
print(f"FINAL BUYER PROFILE: \n\n{user_profile_summary}")

FINAL BUYER PROFILE: 

The human asks the AI for help in finding a house. The AI is happy to assist and suggests discussing the budget for the new home, which is under 800,000. The AI asks about the preferred location for the new home, but the human doesn't have one. The AI then asks how many bedrooms the human is looking for in their new home. The human mentions planning to have kids, so at least 3 bedrooms are needed. The AI acknowledges this and asks about lifestyle preferences and specific features desired in the new home.


In [43]:
profile_prompt = f"""
Based strictly on the following buyer's profile, write a 1-paragraph description of their absolute dream house.
Include specific details about neighborhood vibe, price point, and amenaties mentioned in the profile.

---

BUYER'S PROFILE:

{user_profile_summary}

---

DREAM HOUSE DESCRIPTION:
"""

In [44]:
dream_house_description = llm.predict(profile_prompt)

In [45]:
print(f"GENERATED SEARCH QUERY:\n\n{dream_house_description}")

GENERATED SEARCH QUERY:

The buyer's dream house would be a spacious 3-bedroom home located in a family-friendly neighborhood with a sense of community. With a budget of under 800,000, the ideal home would have modern amenities such as a gourmet kitchen, a backyard for kids to play in, and possibly a home office space for remote work. The neighborhood would offer safe streets for children to ride bikes and play outside, as well as nearby parks and schools. The house would have plenty of natural light, a cozy fireplace for winter nights, and a welcoming front porch. Overall, the dream house would provide a comfortable and inviting space for the buyer's growing family to thrive in.


In [47]:
rag_response = qa_chain.run(dream_house_description)

In [48]:
print(rag_response)

Based on your preferences, I would highly recommend the spacious family home overlooking the serene Lakeview. This property meets all of your criteria, with 3 bedrooms, a large backyard for kids to play in, and a bonus room that can be used as a home office. The cozy fireplace and beautiful view of the lake provide a sense of comfort and tranquility, perfect for your growing family. Additionally, the family-friendly neighborhood offers safe streets, nearby parks, and schools, creating a strong sense of community. With a budget under 800,000, this property is a great fit for you and your family. I believe this home will provide the comfortable and inviting space you are looking for to thrive in. Let's schedule a viewing so you can see it for yourself!
